# 1. 与えられたPDFを元に回答するチャットBotを作成

## 1-1. PDFの読み込み

In [3]:
from langchain.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("sample.pdf")
documents = loader.load()
print(f"ドキュメントの数: {len(documents)}") # ページ数
print(f"1つ目のドキュメントの内容: {documents[0].page_content}") # 最初のページの内容
print(f"1つ目のドキュメントのメタデータ: {documents[0].metadata}") #最初のページのメタデータ

ドキュメントの数: 12
1つ目のドキュメントの内容: 飛行車に関する法制度
【注意】この文章は、架空の飛行車を対象にした法律の自動生成例です。

1つ目のドキュメントのメタデータ: {'source': 'sample.pdf', 'file_path': 'sample.pdf', 'page': 0, 'total_pages': 12, 'format': 'PDF 1.4', 'title': '飛行車に関する法制度', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'macOS バージョン13.5（ビルド22G74） Quartz PDFContext, AppendMode 1.1', 'creationDate': '', 'modDate': "D:20230819044808Z00'00'", 'trapped': ''}


## 1-2. 文章の分割

In [7]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import SpacyTextSplitter

loader = PyMuPDFLoader("sample.pdf")
documents = loader.load()

text_splitter = SpacyTextSplitter(
    chunk_size=300, # 文書を分割するサイズ
    pipeline="ja_core_news_sm" # 分割に使用する言語モデルを選択
)
splitted_documents = text_splitter.split_documents(documents)

print(f"分割後のドキュメント数: {len(splitted_documents)}")

分割後のドキュメント数: 56


## 1-3. 分割した文章をベクトル化し、データベースに保存

In [5]:
from langchain.document_loaders import PyMuPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import SpacyTextSplitter
from langchain.vectorstores import Chroma

### PDF読み込み
loader = PyMuPDFLoader("sample.pdf")
documents = loader.load()

### 文章の分割
text_splitter = SpacyTextSplitter(
    chunk_size=300, # 文書を分割するサイズ
    pipeline="ja_core_news_sm" # 分割に使用する言語モデルを選択
)
splitted_documents = text_splitter.split_documents(documents)

### Embeddingモデルの指定
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

### データベースにEmbedding後のデータを保存
database = Chroma(
    persist_directory="./.data", # 永続化データの保存先を指定
    embedding_function=embeddings
)
database.add_documents(splitted_documents)

print("データベースの作成が完了しました")

データベースの作成が完了しました


## 1-4. ベクトルデータベースで検索を実行

In [7]:
documents = database.similarity_search("飛行機の最高速度は？") # 類似度が近い文章を検索し、Documentをリストで返す

print(f"ドキュメントの数: {len(documents)}")
for document in documents:
    print(f"ドキュメントの内容: {document.page_content}")

ドキュメントの数: 4
ドキュメントの内容: 飛行車速度制限法
第1条（目的）
本法は、飛行車の飛行安全及び一般公共の利益を確保するため、飛行車の飛行速度に関する
基準を定めることを目的とする。




第2条（定義）
本法において「飛行車」とは、地上及び空中を移動する能力を有する車両を指す。




第3条（一般的な速度制限）
1.
都市部において飛行車が飛行する場合の最大速度は、時速150キロメートルとする。


2.
都市部以外の地域において飛行車が飛行する場合の最大速度は、時速250キロメート
ルとする。


3.
特定の地域や施設の上空、または特定の飛行コース上では、別途速度制限が設けられ
ることがある。
ドキュメントの内容: 飛行車速度制限法
第1条（目的）
本法は、飛行車の飛行安全及び一般公共の利益を確保するため、飛行車の飛行速度に関する
基準を定めることを目的とする。




第2条（定義）
本法において「飛行車」とは、地上及び空中を移動する能力を有する車両を指す。




第3条（一般的な速度制限）
1.
都市部において飛行車が飛行する場合の最大速度は、時速150キロメートルとする。


2.
都市部以外の地域において飛行車が飛行する場合の最大速度は、時速250キロメート
ルとする。


3.
特定の地域や施設の上空、または特定の飛行コース上では、別途速度制限が設けられ
ることがある。
ドキュメントの内容: 飛行車速度制限法
第1条（目的）
本法は、飛行車の飛行安全及び一般公共の利益を確保するため、飛行車の飛行速度に関する
基準を定めることを目的とする。




第2条（定義）
本法において「飛行車」とは、地上及び空中を移動する能力を有する車両を指す。




第3条（一般的な速度制限）
1.
都市部において飛行車が飛行する場合の最大速度は、時速150キロメートルとする。


2.
都市部以外の地域において飛行車が飛行する場合の最大速度は、時速250キロメート
ルとする。


3.
特定の地域や施設の上空、または特定の飛行コース上では、別途速度制限が設けられ
ることがある。
ドキュメントの内容: 第3条（一般的な速度制限）
1.
都市部において飛行車が飛行する場合の最大速度は、時速150キロメートルとする。


2.
都市部以外の地域において飛行車

## 1-5. 検索結果と質問を組み合わせて質問に回答

In [12]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain.vectorstores import Chroma

### Embeddingモデルの指定
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

### データベースにEmbedding後のデータを保存
database = Chroma(
    persist_directory="./.data", # 永続化データの保存先を指定
    embedding_function=embeddings
)

### 質問とその類似するDocumentを検索し、変数に格納
query = "飛行車の最高速度は？"
documents = database.similarity_search(query)
documents_string = "" #ドキュメントの内容を格納する変数を初期化
for document in documents:
    documents_string += f"""
    ------------------------------
    {document.page_content}
    """

### プロンプトの設定
prompt = PromptTemplate(
    template="""文章を元に質問に答えてください。

    文章：
    {document}

    質問：{query}
    """,
    input_variables=["document", "query"]
)

### チャットBotの設定と回答の取得
chat = ChatOpenAI(model="gpt-3.5-turbo")
result = chat([
    HumanMessage(content=prompt.format(document=documents_string, query=query)),
])

print(result.content)


都市部においては時速150キロメートル、都市部以外の地域においては時速250キロメートルとなります。


# 2. RetrievalQAを使ってQAシステムの構築を楽にする

In [17]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

### ChatGPTモデルの指定
chat = ChatOpenAI(model="gpt-3.5-turbo")

### Embeddingモデルの指定
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

### データベースを呼び出し、Retrieverに変換
database = Chroma(
    persist_directory="./.data", # 永続化データの保存先を指定
    embedding_function=embeddings
)
retriever = database.as_retriever()

### RetrievalQAを設定
qa = RetrievalQA.from_llm(
    llm=chat, # ChatGPTモデルを指定
    retriever=retriever, # Retrieverを指定
    return_source_documents=True, # 返答にソースドキュメントを含めるかどうかを指定
)

### 質問と回答を取得
result = qa("飛行車の最高速度を教えて")

print(result["result"])
print(result["source_documents"])

飛行車の最高速度は、都市部において時速150キロメートル、都市部以外の地域において時速250キロメートルとなります。
[Document(page_content='飛行車速度制限法\n第1条（目的）\n本法は、飛行車の飛行安全及び一般公共の利益を確保するため、飛行車の飛行速度に関する\n基準を定めることを目的とする。\n\n\n\n\n第2条（定義）\n本法において「飛行車」とは、地上及び空中を移動する能力を有する車両を指す。\n\n\n\n\n第3条（一般的な速度制限）\n1.\n都市部において飛行車が飛行する場合の最大速度は、時速150キロメートルとする。\n\n\n2.\n都市部以外の地域において飛行車が飛行する場合の最大速度は、時速250キロメート\nルとする。\n\n\n3.\n特定の地域や施設の上空、または特定の飛行コース上では、別途速度制限が設けられ\nることがある。', metadata={'author': '', 'creationDate': '', 'creator': '', 'file_path': 'sample.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20230819044808Z00'00'", 'page': 3, 'producer': 'macOS バージョン13.5（ビルド22G74） Quartz PDFContext, AppendMode 1.1', 'source': 'sample.pdf', 'subject': '', 'title': '飛行車に関する法制度', 'total_pages': 12, 'trapped': ''}), Document(page_content='飛行車速度制限法\n第1条（目的）\n本法は、飛行車の飛行安全及び一般公共の利益を確保するため、飛行車の飛行速度に関する\n基準を定めることを目的とする。\n\n\n\n\n第2条（定義）\n本法において「飛行車」とは、地上及び空中を移動する能力を有する車両を指す。\n\n\n\n\n第3条（一般的な速度制限）\n1.\n都市部において飛行車が飛行する場合の最大速度は、時速150キロメートルとする。\n\n\n2.\n都市部以外の地域において飛行車が飛行する場

# 3. 用意されたRetrieversを使ってWikipediaを情報源にする

In [20]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.retrievers import WikipediaRetriever

### ChatGPTモデルの指定
chat = ChatOpenAI(model="gpt-3.5-turbo")

### WikipediaRetrieverを設定
retriever = WikipediaRetriever(
    lang="ja", # 対象言語を指定
    doc_content_chars_max=500, # 取得するテキストの最大文字数を指定
    top_k_results=2, # 検索結果の上位何件を取得するかを指定
)

### RetrievalQAの設定と実行
chain = RetrievalQA.from_llm(
    llm=chat, # ChatGPTモデルを指定
    retriever=retriever, # Retrieverを指定
    return_source_documents=True, # 返答にソースドキュメントを含めるかどうかを指定
)
result = chain("生成AIとは？")

### 結果の表示
source_documents = result["source_documents"]
print(f"検索結果: {len(source_documents)}件")
for document in source_documents:
    print("---------------取得したメタデータ---------------")
    print(document.metadata)
    print("---------------取得したテキスト---------------")
    print(document.page_content[:100])
print("---------------返答---------------")
print(result["result"])

検索結果: 2件
---------------取得したメタデータ---------------
{'title': '人工知能', 'summary': '人工知能（じんこうちのう、英: artificial intelligence）、AI（エーアイ）とは、「『計算（computation）』という概念と『コンピュータ（computer）』という道具を用いて『知能』を研究する計算機科学（computer science）の一分野」を指す語。「言語の理解や推論、問題解決などの知的行動を人間に代わってコンピュータに行わせる技術」、または、「計算機（コンピュータ）による知的な情報処理システムの設計や実現に関する研究分野」ともされる。大学でAI教育研究は、情報工学科や情報理工学科コンピュータ科学専攻などの組織で行われている（工学〔エンジニアリング〕とは、数学・化学・物理学などの基礎科学を工業生産に応用する学問）。\n『日本大百科全書(ニッポニカ)』の解説で、情報工学者・通信工学者の佐藤理史は次のように述べている。\n\n1200の大学で使用された事例がある計算機科学の教科書『エージェントアプローチ人工知能』は、最終章最終節「結論」で、未来はどちらへ向かうのだろうか？と述べて次のように続ける。SF作家らは、筋書きを面白くするためにディストピア的未来を好む傾向がある。しかし今までのAIや他の革命的な科学技術（出版・配管・航空旅行・電話システム）について言えば、これらの科学技術は全て好影響を与えてきた。同時にこれらは不利な階級へ悪影響を与えており、われわれは悪影響を最小限に抑えるために投資するのがよいだろう。論理的限界まで改良されたAIが、従来の革命的技術と違って人間の至高性を脅かす可能性もある。前掲書の「結論」は、次の文で締めくくられている。', 'source': 'https://ja.wikipedia.org/wiki/%E4%BA%BA%E5%B7%A5%E7%9F%A5%E8%83%BD'}
---------------取得したテキスト---------------
人工知能（じんこうちのう、英: artificial intelligence）、AI（エーアイ）とは、「『計算（computation）』という概念と『コンピュータ（computer）』という道

## 3-2. Retrieverでどのような検索を行うか指示する

In [24]:
from langchain_openai import ChatOpenAI
from langchain.retrievers import WikipediaRetriever, RePhraseQueryRetriever
from langchain import LLMChain
from langchain.prompts import PromptTemplate

### WikipediaRetrieverを設定
retriever = WikipediaRetriever(
    lang="ja", # 対象言語を指定
    doc_content_chars_max=500, # 取得するテキストの最大文字数を指定
    top_k_results=2, # 検索結果の上位何件を取得するかを指定
)

### プロンプトを指定
prompt = PromptTemplate(
    input_variables=["question"],
    template = """以下の質問からWikipediaで検索するべきキーワードを抽出してください。
    質問： {question}
    """
)

### LLMChainを指定
llm_chain = LLMChain(
    llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature = 0,
    ),
    prompt=prompt,
)

### 質問を再構築するためのRetrieverを設定
re_phrase_query_retriever = RePhraseQueryRetriever(
    llm_chain=llm_chain,
    retriever=retriever,
)

### 結果の表示
documents = re_phrase_query_retriever.get_relevant_documents("私はうどんが好きです。ところで、生成AIとは何ですか？")
print(documents)

[Document(page_content='人工知能（じんこうちのう、英: artificial intelligence）、AI（エーアイ）とは、「『計算（computation）』という概念と『コンピュータ（computer）』という道具を用いて『知能』を研究する計算機科学（computer science）の一分野」を指す語。「言語の理解や推論、問題解決などの知的行動を人間に代わってコンピュータに行わせる技術」、または、「計算機（コンピュータ）による知的な情報処理システムの設計や実現に関する研究分野」ともされる。大学でAI教育研究は、情報工学科や情報理工学科コンピュータ科学専攻などの組織で行われている（工学〔エンジニアリング〕とは、数学・化学・物理学などの基礎科学を工業生産に応用する学問）。\n『日本大百科全書(ニッポニカ)』の解説で、情報工学者・通信工学者の佐藤理史は次のように述べている。\n\n1200の大学で使用された事例がある計算機科学の教科書『エージェントアプローチ人工知能』は、最終章最終節「結論」で、未来はどちらへ向かうのだろうか？と述べて次のように続ける。SF作家らは、筋書きを面白くするためにディストピア的未来を好', metadata={'title': '人工知能', 'summary': '人工知能（じんこうちのう、英: artificial intelligence）、AI（エーアイ）とは、「『計算（computation）』という概念と『コンピュータ（computer）』という道具を用いて『知能』を研究する計算機科学（computer science）の一分野」を指す語。「言語の理解や推論、問題解決などの知的行動を人間に代わってコンピュータに行わせる技術」、または、「計算機（コンピュータ）による知的な情報処理システムの設計や実現に関する研究分野」ともされる。大学でAI教育研究は、情報工学科や情報理工学科コンピュータ科学専攻などの組織で行われている（工学〔エンジニアリング〕とは、数学・化学・物理学などの基礎科学を工業生産に応用する学問）。\n『日本大百科全書(ニッポニカ)』の解説で、情報工学者・通信工学者の佐藤理史は次のように述べている。\n\n1200の大学で使用された事例がある計算機科学の教科書『エージェントアプローチ人工知能』は

In [27]:
print(f"検索結果: {len(documents)}件")
for document in documents:
    print("---------------取得したメタデータ---------------")
    print(document.metadata)
    print("---------------取得したテキスト---------------")
    print(document.page_content[:200])
# print("---------------返答---------------")
# print(documents["result"])

検索結果: 2件
---------------取得したメタデータ---------------
{'title': '人工知能', 'summary': '人工知能（じんこうちのう、英: artificial intelligence）、AI（エーアイ）とは、「『計算（computation）』という概念と『コンピュータ（computer）』という道具を用いて『知能』を研究する計算機科学（computer science）の一分野」を指す語。「言語の理解や推論、問題解決などの知的行動を人間に代わってコンピュータに行わせる技術」、または、「計算機（コンピュータ）による知的な情報処理システムの設計や実現に関する研究分野」ともされる。大学でAI教育研究は、情報工学科や情報理工学科コンピュータ科学専攻などの組織で行われている（工学〔エンジニアリング〕とは、数学・化学・物理学などの基礎科学を工業生産に応用する学問）。\n『日本大百科全書(ニッポニカ)』の解説で、情報工学者・通信工学者の佐藤理史は次のように述べている。\n\n1200の大学で使用された事例がある計算機科学の教科書『エージェントアプローチ人工知能』は、最終章最終節「結論」で、未来はどちらへ向かうのだろうか？と述べて次のように続ける。SF作家らは、筋書きを面白くするためにディストピア的未来を好む傾向がある。しかし今までのAIや他の革命的な科学技術（出版・配管・航空旅行・電話システム）について言えば、これらの科学技術は全て好影響を与えてきた。同時にこれらは不利な階級へ悪影響を与えており、われわれは悪影響を最小限に抑えるために投資するのがよいだろう。論理的限界まで改良されたAIが、従来の革命的技術と違って人間の至高性を脅かす可能性もある。前掲書の「結論」は、次の文で締めくくられている。', 'source': 'https://ja.wikipedia.org/wiki/%E4%BA%BA%E5%B7%A5%E7%9F%A5%E8%83%BD'}
---------------取得したテキスト---------------
人工知能（じんこうちのう、英: artificial intelligence）、AI（エーアイ）とは、「『計算（computation）』という概念と『コンピュータ（computer）』という道